<a href="https://colab.research.google.com/github/muhammadanas25/Stock_trend_prediction/blob/main/Gradient_Boosting_Model__Stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Initial imports
import pandas as pd
!pip install path
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
# Initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

from sklearn.compose import make_column_selector as selector


In [2]:
# Loading data
file_path = Path("IBM.csv")
df = pd.read_csv(file_path)
df.dropna(inplace=True)
df=df[df.FutureTrend !='Neutral']
df.tail()

,Date,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
749,23/06/2021,146.429993,146.500000,144.570007,144.610001,3202600,0.020722,-1.242909,Negative,Positive,Negative,0,Negative
750,24/06/2021,145.800003,146.190002,144.690002,145.440002,3609700,0.500000,-0.246914,Negative,Negative,Negative,0,Negative
751,25/06/2021,145.380005,146.929993,144.949997,146.839996,3594200,0.954547,1.004259,Positive,Negative,Positive,0,Positive
752,28/06/2021,147.009995,147.300003,144.910004,145.289993,3888900,0.158992,-1.169989,Negative,Positive,Negative,0,Negative
753,29/06/2021,145.259995,146.740005,145.100006,145.550003,2428500,0.274388,0.199648,Negative,Negative,Positive,0,Negative


In [3]:
#OHLC Chart with Volume
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Ohlc(x=df.Date,
                      open=df.Open,
                      high=df.High,
                      low=df.Low,
                      close=df.Close,
                      name='Price'), row=1, col=1)

fig.add_trace(go.Scatter(x=df.Date, y=df.Volume, name='Volume'), row=2, col=1)

fig.update_layout(title_text="HPQ Volume")
fig.update(layout_xaxis_rangeslider_visible=False) 
fig.show()

In [4]:
target=df['FutureTrend']
df=df.drop("FutureTrend",axis =1)
target= target.replace(['Negative','Positive',  'Neutral'],[0,1,2])
df= df.replace(['Negative','Positive',  'Neutral'],[0,1,2])
appl_df =df.set_index('Date')
from imblearn.over_sampling import SMOTE
# pd.DataFrame(oe_df)
oversample = SMOTE()
appl_df, target = oversample.fit_resample(appl_df, target)


In [5]:
new_df=appl_df
new_df

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(new_df)
categorical_columns = categorical_columns_selector(new_df)
print(categorical_columns)
preprocessor = ColumnTransformer([
  # ('one-hot-encoder', categorical_preprocessor, categorical_columns),
   ('standard-scaler', numerical_preprocessor, numerical_columns)
    ])

model_2 = make_pipeline(preprocessor,  GradientBoostingClassifier(n_estimators=38,
                                      learning_rate=0.3,
                                      max_features=6,
                                      max_depth=5,
                                      random_state=4,subsample=1,
                                      criterion='friedman_mse'))
                        
print(new_df.shape,target.shape)
X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2, random_state=42)
model_2.fit(X_train,y_train)

  # Scoring the model_2
print("Accuracy score (training): {0:.3f}".format(
        model_2.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model_2.score(
            X_test, 
            y_test)))

[]
(798, 11) (798,)
Accuracy score (training): 1.000
Accuracy score (validation): 0.525


In [6]:
new_df=appl_df
new_df

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(new_df)
categorical_columns = categorical_columns_selector(new_df)
print(categorical_columns)
preprocessor = ColumnTransformer([
  # ('one-hot-encoder', categorical_preprocessor, categorical_columns),
   ('standard-scaler', numerical_preprocessor, numerical_columns)
    ])

model_1 = make_pipeline(preprocessor,  RandomForestClassifier(n_estimators=70,
                                     
                                      max_features=8,
                                      max_depth=5,
                                      random_state=42))
X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2, random_state=42)
model_1.fit(X_train,y_train)

print("Accuracy score (training): {0:.3f}".format(
        model_1.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model_1.score(
            X_test, 
            y_test)))
                        

[]
Accuracy score (training): 0.818
Accuracy score (validation): 0.550


In [7]:
%pip install lgbm
import lightgbm as lgbm
new_df=appl_df
new_df

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(new_df)
categorical_columns = categorical_columns_selector(new_df)
print(categorical_columns)
preprocessor = ColumnTransformer([
  # ('one-hot-encoder', categorical_preprocessor, categorical_columns),
   ('standard-scaler', numerical_preprocessor, numerical_columns)
    ])

model_lgbm = make_pipeline(preprocessor, lgbm.LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=70,
                                learning_rate=0.4, n_estimators=200, max_depth=4,
                                bagging_fraction=0.6, feature_fraction=0.8,reg_alpha=0.7
                                ,reg_lambda=0.6))
X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2,random_state=42)
model_lgbm.fit(X_train,y_train)
print("Accuracy score (training): {0:.3f}".format(
        model_lgbm.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model_lgbm.score(
            X_test, 
            y_test)))

                        

ERROR: Could not find a version that satisfies the requirement lgbm (from versions: none)
ERROR: No matching distribution found for lgbm
[]
Accuracy score (training): 1.000
Accuracy score (validation): 0.550


In [8]:
X_train

,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments
264,150.570007,150.720001,148.380005,150.380005,3742200,0.854702,-0.126189,1,0,0,613
615,126.349998,126.930000,124.940002,124.959999,4803200,0.010049,-1.100118,1,0,0,118
329,133.520004,134.570007,132.699997,134.380005,3610900,0.898395,0.644099,0,0,1,958
342,137.199997,137.199997,135.000000,135.470001,2568000,0.213637,-1.260930,1,1,0,546
394,139.500000,140.470001,138.750000,139.550003,3451600,0.465118,0.035844,1,0,1,648
...,...,...,...,...,...,...,...,...,...,...,...
71,142.619995,144.190002,138.779999,139.020004,8710500,0.044363,-2.524184,0,1,0,870
106,121.660004,124.690002,121.480003,124.269997,7251600,0.869157,2.145317,0,0,1,578
270,149.699997,150.179993,146.389999,148.240005,3722900,0.488129,-0.975278,1,1,0,651
435,106.910004,113.150002,105.570000,112.889999,7153500,0.965699,5.593486,1,1,1,499


In [9]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

  
seed = 8
kfold = model_selection.KFold(n_splits = 5,
                       )
  
# initialize the base classifier
#base_cls = DecisionTreeClassifier()
  
# no. of base classifier
num_trees = 500
for num_trees in [50,100,250,300,400,500]:
# bagging classifier
  model_3 = BaggingClassifier(base_estimator =GradientBoostingClassifier(n_estimators=38,
                                      learning_rate=0.3,
                                      max_features=6,
                                      max_depth=5,
                                      random_state=4,subsample=1,
                                      criterion='friedman_mse'),
                            n_estimators = num_trees,
                            random_state = seed)
  #X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2, random_state=42)

  model_3.fit(X_train,y_train)
  print("Accuracy score (training): {0:.3f}".format(
        model_3.score(
            X_train, 
            y_train)))
  print("Accuracy score (validation): {0:.3f}".format(
        model_3.score(
            X_test, 
            y_test)))
  # results = model_selection.cross_val_score(model_3, X_train, y_train, cv = kfold)
  # print("accuracy :")
  # print("num_trees :",num_trees,results.mean())

Accuracy score (training): 1.000
Accuracy score (validation): 0.544
Accuracy score (training): 0.997
Accuracy score (validation): 0.550
Accuracy score (training): 1.000
Accuracy score (validation): 0.537
Accuracy score (training): 1.000
Accuracy score (validation): 0.544
Accuracy score (training): 1.000
Accuracy score (validation): 0.537
Accuracy score (training): 1.000
Accuracy score (validation): 0.537


Hyper parameter tuning 

In [10]:
param_grid = {
    'base_estimator__max_depth' : [1, 2, 3, 4, 5],
    'max_samples' : [0.05, 0.1, 0.2, 0.5],
    'n_estimators' : [5,10,20,40,60],
    'bootstrap' :[True,False]

}


#Fitting Bagging Classifier model with  hyper parameters
# bagg = BaggingClassifier(bootstrap=True,n_estimators=200)
# gridbag = GridSearchCV(bagg, param_grid=param_grid, cv = 3, verbose = 1)
# bestbag = gridbag.fit(X_train, y_train)

gridbag = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(),
                                     n_estimators = 250, max_features = 0.5),
                   param_grid)
gridbag.fit(X_train, y_train)
print("Accuracy score (training): {0:.3f}".format(
        gridbag.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        gridbag.score(
            X_test, 
            y_test)))


Accuracy score (training): 0.702
Accuracy score (validation): 0.531


In [11]:
#Printing the best hyperparameters
print('The best hyper parameters are:\n',gridbag.best_params_)



#Checking different metrics for bagging model after tuning the hyperparameters
print('Checking with the best hyper parameter on bagging model after tuning the hyperparameters:\n')

bestbag=BaggingClassifier(bootstrap= False, max_samples=0.5, n_estimators= 40,max_features=0.6
                          ,random_state=45
                          )
bestbag.fit(X_train,y_train)
print("Accuracy score (training): {0:.3f}".format(
        bestbag.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        bestbag.score(
            X_test, 
            y_test)))


The best hyper parameters are:
 {'base_estimator__max_depth': 3, 'bootstrap': True, 'max_samples': 0.5, 'n_estimators': 60}
Checking with the best hyper parameter on bagging model after tuning the hyperparameters:

Accuracy score (training): 0.998
Accuracy score (validation): 0.519


TRY Run2(taking too much time)

In [12]:
# parameters = {
#     "loss":["deviance"],
#     "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
#     "min_samples_split": np.linspace(0.1, 0.5, 12),
#     "min_samples_leaf": np.linspace(0.1, 0.5, 12),
#     "max_depth":[3,5,8],
#     "max_features":["log2","sqrt"],
#     "criterion": ["friedman_mse",  "mae"],
#     "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
#     "n_estimators":[10,20,40,50]
#     }
# from sklearn.compose import make_column_selector as selector
# numerical_columns_selector = selector(dtype_exclude=object)
# categorical_columns_selector = selector(dtype_include=object)
# numerical_columns = numerical_columns_selector(new_df)
# categorical_columns = categorical_columns_selector(new_df)
       
# preprocessor = ColumnTransformer([
#     ('one-hot-encoder', categorical_preprocessor, categorical_columns),
#     ('standard-scaler', numerical_preprocessor, numerical_columns)])


# new_df=preprocessor.fit_transform(new_df)
# model_gbm=GradientBoostingClassifier()
# clf = GridSearchCV(model_gbm, parameters,refit=False,cv=2, n_jobs=-1)

# clf.fit(X_train, y_train)
# print("Accuracy score (training): {0:.3f}".format(
#         clf.score(
#             X_train, 
#             y_train)))
# print("Accuracy score (validation): {0:.3f}".format(
#         clf.score(
#             X_test, 
#             y_test)))